In [3]:
#필요 패키지 import
import openai
import pandas as pd
import time
import re

In [22]:
%store -r voter
%store -r voter_default
%store -r news_list_200
%store -r news_df_200

In [5]:
news_df = news_df_200
news_df

,제목_1,날짜_1,링크_1,내용_1,제목_2,날짜_2,링크_2,내용_2,제목_3,날짜_3,링크_3,내용_3,제목_4,날짜_4,링크_4,내용_4
0,"용인대, 한국서비스경영학회 추계학술대회 우수발표 논문상 수상",2021.12.06.,http://www.newsis.com/view/?id=NISX20211206_00...,,None,None,None,None,None,None,None,None,None,None,None,None
1,"충북대, 'K-MOOC'로 개발완료 강좌 결과보고회",2017.08.21.,http://www.veritas-a.com/news/articleView.html...,"충북대, 'K-MOOC'로 개발완료 강좌 결과보고회 < 대학뉴스 < 대학 < 기사본...","충북대, 교직원 대상 '정부 3.0과 대학행정 활용' 교육",2016.08.31.,http://news.unn.net/news/articleView.html?idxn...,"충북대, 교직원 대상 ‘정부 3.0과 대학행정 활용’ 교육 < 대학뉴스 < 뉴스 <...","한국정책학회, 지역화폐 주제 토론회 개최",2021.12.02.,https://zdnet.co.kr/view/?no=20211202221333,"한국정책학회, 지역화폐 주제 토론회 개최 - ZDNet koreaZDNet Kore...",‘지역화폐 정책 이대로 좋은가’…정책학회 토론회 개최,2021.12.02.,http://www.dt.co.kr/contents.html?article_no=2...,‘지역화폐 정책 이대로 좋은가’…정책학회 토론회 개최 - 디지털타임스구독 ▶네이버 ...
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,동아대 '2022년 SW전문인재양성사업' 선정,2022.06.14.,http://www.veritas-a.com/news/articleView.html...,동아대 '2022년 SW전문인재양성사업' 선정 < 대학뉴스 < 대학 < 기사본문 -...,동아대 바이오헬스융합연구소 '의료/바이오/헬스 분야의 데이터사이언스 기술...,2020.11.24.,http://www.veritas-a.com/news/articleView.html...,동아대 바이오헬스융합연구소 '의료/바이오/헬스 분야의 데이터사이언스 기술 적용' 세...,부산 지산학협력 67호 브랜치 ㈜에스위너스에 개소,2023.09.13.,https://www.busaneconomy.com/news/articleView....,부산 지산학협력 67호 브랜치 ㈜에스위너스에 개소 < 경제 < 뉴스 < 기사본문 -...,"동아대, '2022년 SW전문인재양성사업' 선정",2022.06.16.,http://www.iusm.co.kr/news/articleView.html?id...,"동아대, '2022년 SW전문인재양성사업' 선정 < 부산 < 부산·경남 < 뉴스 <..."
4,"경북대 연구팀 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾아",2023.08.31.,http://www.veritas-a.com/news/articleView.html...,"경북대 연구팀 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾아 < 대학뉴...","경북대, 특허청 '지식재산 전문인력양성 중점대학' 사업 선정",2023.02.23.,http://www.kyongbuk.co.kr/news/articleView.htm...,"경북대, 특허청 '지식재산 전문인력양성 중점대학’ 사업 선정 - 경북일보 - 굿데이...",집먼지진드기로 인한 아토피피부염 치료법 국내 연구진 개발,2023.09.04.,https://kizmom.hankyung.com/news/view.html?aid...,집먼지진드기로 인한 아토피피부염 치료법 국내 연구진 개발 | Kizmom 뉴스 주...,"한국연구재단, 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾았다",2023.08.31.,https://www.jeonmae.co.kr/news/articleView.htm...,"한국연구재단, 집먼지진드기 원인 아토피피부염, 새로운 치료제 실마리 찾았다 - 전국..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
96,"[대학가 투데이] 경희대, 건국대, 인하대, 광운대, 동아대",2022.05.10.,http://www.dhnews.co.kr/news/articleView.html?...,"[대학가 투데이] 경희대, 건국대, 인하대, 광운대, 동아대대학저널포항20.9℃통영...","[팩트체크] ""윤씨""...일본 언론 윤석열 대통령 하대했다?",2023.03.20.,https://www.newstof.com/news/articleView.html?...,"[팩트체크] ""윤씨""...일본 언론 윤석열 대통령 하대했다? < 팩트체크 < 정치 ...","위덕대, 문학 속의 원자력을 다룬 포럼 ‘원자력과 문학’ 개최",2022.11.30.,http://www.breaknews.com/936414,"위덕대, 문학 속의 원자력을 다룬 포럼 ‘원자력과 문학’ 개최시작페이지로 l 즐겨찾...","위덕대, 문학 속의 원자력 다룬 포럼 ‘원자력과 문학’ 개최",2022.11.30.,https://www.wikitree.co.kr/articles/811154,"위덕대, 문학 속의 원자력 다룬 포럼 ‘원자력과 문학’ 개최 | 위키트리위키트리 ..."
97,"충북대 보안컨설팅 융합전공 학생들, '보건의료 워킹그룹 모의 해킹메일 훈련...",2021.12.30.,http://www.veritas-a.com/news/articleView.html...,"충북대 보안컨설팅 융합전공 학생들, ‘보건의료 워킹그룹 모의 해킹메일 훈련’대회에서...","충북대 학생팀, 'HIRA 빅데이터 기반 산·학·관 연계 맞춤형 인재 양성 교육...",2020.09.04.,http://www.veritas-a.com/news/articleView.html...,"충북대 학생팀, 'HIRA 빅데이터 기반 산·학·관 연계 맞춤형 인재 양성 교육' ...","사이버위협 능동대응 기술 개발에 총 5년간 3,917억 원 투자 추진",2023.05.16.,https://www.dailysecu.com/news/articleView.htm...,"사이버위협 능동대응 기술 개발에 총 5년간 3,917억 원 투자 추진 - 데일리시큐...","제11회 CPS 보안워크숍, 9월 22일~23일 개최",2022.09.14.,https://www.dailysecu.com/news/articleView.htm...,"제11회 CPS 보안워크숍, 9월 22일~23일 개최 - 데일리시큐 ×전체기사산업이..."
98,서울대 교수들도 “집필 않겠다”…주요대학 모두 거부,A5면 TOP,http://www.hani.co.kr/arti/society/schooling/7...,,None,None,None,None,None,None,None,None,None,None,None,None


In [6]:
voters = voter_default[100:200].drop(['임시직위'], axis = 1)
voters.index = [x for x in range(100)]
voters

,이름,소속,직위,이메일,핸드폰
0,김상수,용인대학교,조교수,sgssk@naver.com,010-3234-5650
1,김상욱,충북대학교,교수,sierra@chungbuk.ac.kr,010-5465-4432
2,김상원,국민대학교,학생,kimsangwon@kookmin.ac.kr,010-8914-9620
3,김상진,동아대학교,교수,skim10@dau.ac.kr,010-3034-0129
4,김상현,경북대학교,교수,ksh@knu.ac.kr,010-6716-1836
...,...,...,...,...,...
95,김치헌,신한카드,부부장,chehukim@naver.com,010-4198-3250
96,김태경,경희대학교,부교수,kimtk@kw.ac.kr,010-7279-1994
97,김태성,충북대학교,정교수,kimts@cbnu.ac.kr,010-5391-2468
98,김태웅,성균관대학교,교수(은퇴),tukim@skku.edu,010-9025-0452


In [ ]:
df = pd.concat([voters, news_df], axis=1, join= 'outer')
df

In [ ]:
%store -r profile_df_default
profile_df = profile_df_default[100:200]
profile_df.index = [x for x in range(100)]
profile_df

In [ ]:

# 발급받은 API 키 설정
OPENAI_API_KEY = 
openai.api_key = OPENAI_API_KEY

In [10]:
#ChatGPT로 프롬프트 보냄
#"gpt-3.5-turbo-16k"
#"gpt-4"
def get_completion(x, temperature=0):
    model = x
    messages = prompt
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

---

In [8]:
#프로필 생성하는 함수
def get_profile(x, y):
  global profile
  profile = f"""
      Based on the text below, wirte down {x}'s profile.
      - Please write down "affiliated"
      - Please write down "detail affiliated" such as "경영대학원", "데이터사이언스학과"
      - Please write down 'whether his/her affiliated is0 metropolitan or non-metropolitan areas':
        this should be written only '수도권' or '비수도권', only if it is his/her affiliated is a university in Korea.
      - Please write "academic background"
        if there are various academic backgrounds, write only where he or she got the doctor degree.
      - Please write down his or her major or research field in short keywords.
      - If there are any other keywords for {x}, please organize them.
      You will write them in the following format.

      소속:
      세부소속:
      지역:
      학력:
      연구분야:
      기타 키워드:
      
      Message will write in Korean
      Please do not write other person's information.
      If you can't, please write X.
      """

  global contents
  contents = y

  global prompt
  prompt = [{"role" : "system",
              "content" : profile},
            {"role" : "user",
            "content" : f'{contents}'}]
  return prompt

In [ ]:
#프로필 생성
#오류 났을 경우 반드시 gpt_profile 저장 후 재시도할 것
gpt_profile = []
for i in range(len(voters)):
    %store gpt_profile
    try:
        get_profile(voter.iloc[i, 0], prof_total[i])
        print(voter.iloc[i, 0] + '의 정보는 다음과 같습니다.')
        gpt_profile.append(get_completion("gpt-3.5-turbo-16k"))
        print(gpt_profile[i])
        print('--------------------------------------------------------------------------------------')
    #except openai.error.InvalidRequestError: #토큰 수 초과될 경우
    #    print('Invalid Request Error')
    #    summarize(voter.iloc[i, 0], prof_total[i])
    #    response = openai.ChatCompletion.create(model="gpt-3.5-turbo-16k", messages=messages)
    #    get_profile(voter.iloc[i, 0], response.choices[0].message["content"])
    #    gpt_profile.append(get_completion())
    #    print(gpt_profile[i])
    #    print('-------------------------------------------------------------------')
    except: #(대체로) 1분에 너무 많은 토큰 불러올 경우: 60초 후 재시도
        #try:
            print('retry in 60s')
            time.sleep(60)
            get_profile(voter.iloc[i, 0], prof_total[i])
            gpt_profile.append(get_completion("gpt-3.5-turbo-16k"))
            print(gpt_profile[i])
            print('--------------------------------------------------------------------------------------')
        #except openai.error.InvalidRequestError:
            #print('Invalid Request Error')
            #summarize(voter.iloc[i, 0], prof_total[i])
            #response = openai.ChatCompletion.create(model="gpt-3.5-turbo-16k", messages=messages)
            #get_profile(voter.iloc[i, 0], response.choices[0].message["content"])
            #gpt_profile.append(get_completion())
            #print(gpt_profile[i])
            #print('-------------------------------------------------------------------')


In [ ]:
new_profile = [[x] for x in voters['이름']+" "+voters['소속']]

In [ ]:
#GPT OUTPUT으로부터 각 카테고리별 정리
def categorize(x):
    for n in range(len(gpt_profile)):
        a = gpt_profile[n].find(x)
        b = gpt_profile[n].find('\n', a)
        c = gpt_profile[n][a+len(x)+2:b]

        if c == 'X':
            new_profile[n].append('X')
        elif len(re.split(',', c)) == 1 and c != 'X':
            new_profile[n].append(re.split(',', c)[0])
        else:
            new_profile[n].append(re.split(',', c)[0:2])

In [ ]:
categorize('세부소속')
categorize('지역')
categorize('학력')
categorize('연구분야')
categorize('기타 키워드')

In [ ]:
#생성한 프로필을 데이터프레임으로
df1 = pd.DataFrame(data=new_profile, columns=['a', '세부소속', '위치', '학력', '관심분야', '기타 키워드'])
df1 = df1.drop(['a'], axis=1)
voters = voters.drop(['임시직위'], axis = 1).reset_index(drop=True)
df1 = pd.concat([voters, df1], axis = 1, join= 'outer')
df1

In [83]:
backup_gpt = gpt_profile
new_profile = [[x] for x in voter['이름']+" "+voter['소속']]

NameError: name 'gpt_profile' is not defined

In [ ]:
#GPT OUTPUT으로부터 각 카테고리별 정리
def categorize(x):
    for n in range(len(gpt_profile)):
        a = gpt_profile[n].find(x)
        b = gpt_profile[n].find('\n', a)
        c = gpt_profile[n][a+len(x)+2:b]

        if c == 'X':
            new_profile[n].append('X')
        elif len(re.split(',', c)) == 1 and c != 'X':
            new_profile[n].append(re.split(',', c)[0])
        else:
            new_profile[n].append(re.split(',', c)[0:2])

In [ ]:
categorize('세부소속')
categorize('지역')
categorize('학력')
categorize('연구분야')
categorize('기타 키워드')

In [ ]:
#생성한 프로필을 데이터프레임으로
df1 = pd.DataFrame(data=new_profile, columns=['a', '세부소속', '위치', '학력', '관심분야', '기타 키워드'])
df1 = df1.drop(['a'], axis=1)
voters = voter.drop(['임시직위'], axis = 1).reset_index(drop=True)
df1 = pd.concat([voters, df1], axis = 1, join= 'outer')
df1

,이름,소속,직위,세부소속,위치,학력,관심분야,기타 키워드
0,박아름,용인예술과학대학교,조교수,빅데이터경영과,수도권,경희대학교 경영학과 박사,"[빅데이터, 디지털 마케팅]","[사물인터넷 서비스 기획, 데이터 사이언스]"
1,박영기,George Washington University,Associate Professor,X,비수도권,X,X,"[한국경영정보학회, 종신회]"
2,박윤주,서울과학기술대학교,조교수,글로벌테크노경영전공,수도권,한국과학기술원(KAIST) 경영대학 박사,"[MIS, 데이터마이닝]","[삼성생명 기획관리실 정보기획P, New York University]"
3,박재홍,영진전문대학교,교수,영진전문대학교 간호학과,비수도권,X,간호학,
4,박정선,명지대학교,교수,X,X,X,X,
...,...,...,...,...,...,...,...,...
513,황재훈,연세대학교,교수,경영학부,수도권,미국 네브래스카주립대학교 대학원 경영정보학 박사,ERP시스템,
514,황하진,대구가톨릭대학교,교수(은퇴?_),경영정보학과,비수도권,미국 미시시피주립대 경영정보학 박사,"[지식경영, e-비즈니스 모델]","[세계인명사전 등재, 미네소타주립대 부교수]"
515,황현덕,한국건설기술연구원,수석행정원,X,수도권,X,X,
516,황현석,한림대학교,교수,X,비수도권,포항공과대학교(공학박사-산업경영공학과),"[산업경영공학, 빅데이터 분석]","[한국정보시스템학회 상임이사, 한국지식경영학회 이사]"


---

In [82]:
#뉴스기사 요약하는 함수
#w: 날짜, #x: 이름, y: 기사, z: 소속
def get_news(w, x, y, z):
  global p_news
  p_news = f"""
      Message will be written in Korean.
      Based on the news, wirte down what {x} did. Please write it in 1~2 sentences.
      Add {w} in front of the sentence if you can.
      Please judge if {x} in the news belongs to {z}. If not, do not write the senteces and write 'X' only.
      - It is likely that {x} belongs to {z} if you can find {z} and {x} together in one sentence.
      If the news does not contain what {x} did, write 'X' only. In this case, you don't need to add {w}
      """

  global news
  news = y

  global prompt
  prompt = [{"role" : "system",
              "content" : p_news},
            {"role" : "user",
            "content" : f'{news}'}]
  return prompt

In [81]:
#c ="한동대, 우즈벡 국립경영대학원에서 공동교육과정 시행 < 대학/교육 < 종합뉴스 < 기사본문 - 대경일보  주요서비스 바로가기본문 바로가기매체정보 바로가기로그인 바로가기기사검색 바로가기전체서비스 바로가기상단영역최종편집 : 2023-10-23 09:10 (월) 로그인회원가입제보facebookbloginstagram전체 메뉴종합뉴스정치경제사회대학/교육문화/건강의료/복지스포츠연예국제지역뉴스대구포항구미경주경산안동김천칠곡영주상주영천문경의성울진성주예천청도영덕고령봉화청송군위영양울릉기타기획특집오피니언사설칼럼기고기자수첩일반보훈칼럼사람들동정사람들만나고 싶은 사람들인사/부고/화촉포토뉴스영상뉴스커뮤니티공지사항자유게시판자료실제보지면보기검색기사검색검색본문영역  이전 기사보기다음 기사보기한동대, 우즈벡 국립경영대학원에서 공동교육과정 시행바로가기복사하기본문 글씨 줄이기본문 글씨 키우기스크롤 이동 상태바현재위치 홈종합뉴스대학/교육한동대, 우즈벡 국립경영대학원에서 공동교육과정 시행기자명김민지 기자 입력 2020.01.19 19:38 댓글 0바로가기복사하기본문 글씨 줄이기본문 글씨 키우기SNS 기사보내기페이스북(으)로 기사보내기트위터(으)로 기사보내기카카오스토리(으)로 기사보내기카카오톡(으)로 기사보내기구글+(으)로 기사보내기네이버밴드(으)로 기사보내기URL복사(으)로 기사보내기이메일(으)로 기사보내기다른 공유 찾기기사스크랩하기 이 기사를 공유합니다페이스북(으)로 기사보내기트위터(으)로 기사보내기카카오스토리(으)로 기사보내기카카오톡(으)로 기사보내기구글+(으)로 기사보내기네이버밴드(으)로 기사보내기URL복사(으)로 기사보내기닫기우즈벡 국립경영대학원과 사업 협의를 통한 고등교육 발전 방향 모색▲ 한동대 및 건국대 교수와 국립경영대학원 교수들이 학술적 수월성을 위한 워크숍을 진행하고 있다. /한동대학교 제공한동대학교는 지난 13일부터 17일까지 우즈베키스탄 타슈켄트 소재 국립경영대학원에서 국영 기업 공무원 및 대학원생들을 대상으로 공동교육과정을 시행했다.이번 공동교육과정에는 한동대 이종철, 지범하, 안진원 교수와 건국대학교 김용재 교수가 참여해 ‘Joint Workshop on Academic Excellence’라는 주제로 특강, 논문지도 및 워크숍 등을 진행했다. 이번 공동교육과정은 시장 경제로 전환한 경제 현장에서 직면할 수 있는 주요 문제들을 다루는 관점과 문제 해결 교육을 제공함으로써 참가자들의 문제 해결 역량을 강화하고자 진행됐다.한동대는 또한  Rustam Abduraupov 국립경영대학원장과 회의를 통해 현재까지 양교 교류·협력 현황을 확인하고, 우즈벡 발전을 위한 노력에 대해 논의했다. Abduraupov 원장은 2019년도까지 공동교육과정 진행 과정과 2020년도 협력 강화 방안에 대해 의견을 나누며 “지속해서 친밀하고 전문적인 교류·협력을 통해 학생들에게 한동대학교의 강점인 국제적인 시야를 제공하고 교수진들의 지혜와 지식을 공유함으로써, 국제무대에서 경쟁력 있는 우즈벡 경영인으로 성장할 수 있기를 희망한다”라고 말했다. 지범하 한동대 교수는 “연간 2회씩 우즈벡을 방문하며 전문가 파견, 국제 컨퍼런스 개최, 특강 및 워크숍 진행 등 우즈벡의 미래 인재 양성을 위해 다방면으로 노력해왔으며, 우즈벡의 인재들이 새로운 산업을 창출함으로써 우즈벡의 혁신적인 경제 성장을 이끌 수 있도록 지속적인 협력 과정을 통해 창의적인 문제 해결 능력 함양과 전문적 지식을 제공하기 위해 최선을 다할 것”이라고 협력 강화 의지를 밝혔다. 이번 공동교육과정에 참가한 대학원생 Nurim Berdimuratov 씨는 “공동교육과정에 참여함으로써 우즈벡이 직면하고 있는 다양한 이슈와 관련된 문제 해결 역량에 대해 배웠으며, 이를 기반으로 경제발전을 추구하고 있는 우즈벡이 넘어야 할 장벽들에 대한 이해와 새로운 관점을 얻을 수 있었다”라고 말했다. 김민지 기자10hyacinth@naver.com다른기사 보기 저작권자 © 대경일보 무단전재 및 재배포 금지당신만 안 본 뉴스홍준표 “협약파기” 거짓 논리… 희생양 된 김장호 대노[단독] 대구경북영어마을 150억원대 '보조금 부정'  경찰 수사단풍 남하··· 대구경북 19~25일 시작[4·10총선] 포항북구 국힘 공천 ‘4파전’ 양상리뉴얼된 참소주 모델 , 가수 이찬원 발탁공공기관 47개, 입사하면 특정 노조 가입 강제?경산 중산자이 무더기 하자 발견…입주예정자들 “사전점검 무효” 반발홍준표 “협약파기” 거짓 논리… 희생양 된 김장호 대노댓글삭제삭제한 댓글은 다시 복구할 수 없습니다.그래도 삭제하시겠습니까?비밀번호 삭제닫기기사 댓글0 댓글 접기로그인 후 이용 가능합니다.댓글 내용입력0 / 400  비회원 로그인이름비밀번호댓글 내용입력0 / 400 등록댓글쓰기계정을 선택하시면 로그인·계정인증을 통해 댓글을 남기실 수 있습니다.회원 로그인비회원 글쓰기이름비밀번호자동등록방지 확인 취소로그인 옵션 창닫기주요기사“인백기천 자세로 다가올 25년 준비하자”“구미시·기업 힘 합쳐 문화 활성화 이루자”쉬는 의사·인력난 병원 매칭 인력뱅크 추진경북교육청 전국체전 3회 연속 종합 3위포항 장성성당 30주년 기념 한마음 체육대회 성황이강덕 시장 “포항 수소연료전지 기술 개발 협력해주길”최신뉴스영천마늘 16.5t 미국 첫 수출 선적경북도, 중앙아시아 최대 수출시장 우즈벡 시장 가능성 봤다대구경북경제자유구역청(DGFEZ), 2023 생활소비재 소재 산업 세미나 개최포스코 WTP강 소재 산불 진화 보조장비 개발“인백기천 자세로 다가올 25년 준비하자”포토뉴스[카메라 고발] 도로 점령한 건축 폐자재물롯데백화점, 랩그로운 다이아몬드 '루미너스랩' 팝업 진행국감 선서하는 임종식 경북교육감김천 어모에서 화사한 가을을 만끽하세요!인기뉴스1홍준표 “협약파기” 거짓 논리… 희생양 된 김장호 대노2[단독] 대구경북영어마을 150억원대 '보조금 부정'  경찰 수사3단풍 남하··· 대구경북 19~25일 시작4[4·10총선] 포항북구 국힘 공천 ‘4파전’ 양상5리뉴얼된 참소주 모델 , 가수 이찬원 발탁6공공기관 47개, 입사하면 특정 노조 가입 강제?7경산 중산자이 무더기 하자 발견…입주예정자들 “사전점검 무효” 반발8포항 KTX신도시지구 개발, 절차 내팽개친 ‘졸속’ 의혹  하단영역하단메뉴매체소개윤리강령편집규약독자권익위원회고충처리인기사제보광고문의이용약관개인정보처리방침청소년보호정책저작권보호정책이메일무단수집거부서비스 전체보기매체정보본사 경상북도 포항시 남구 새천년대로 408 대표전화 : 054-274-7070팩스 : 054-273-0066대구지사 대구시 서구 문화로 237-1, 3층전화 : 053-746-0071팩스 : 053-655-5758북부본부 경상북도 영주시 조암로 306전화 : 054-633-5880  팩스 : 054-634-5880제호 : 대경일보등록번호 : 경북 아 00728등록일 : 2006-02-01발행·편집인 : 허경태청소년보호책임자 : 최영열대경일보 모든 콘텐츠(영상,기사, 사진)는 저작권법의 보호를 받는 바, 무단 전재와 복사, 배포 등을 금합니다.Copy"
#get_news('2020.01.19', '김용재', c, '건국대')
#get_completion("gpt-4")

"2020.01.19 건국대학교 김용재 교수는 우즈베키스탄 타슈켄트 소재 국립경영대학원에서 한동대학교 교수들과 함께 공동교육과정을 시행하였다. 이 과정에서 김용재 교수는 'Joint Workshop on Academic Excellence'라는 주제로 특강, 논문지도 및 워크숍 등을 진행하였다."

In [55]:
gpt_news = []
for i in range(len(voters)):
    gpt_news.append([])

In [25]:
news_list = news_list_200

In [26]:
#각 뉴스기사들 요약
for n in range(len(voters)):
    %store gpt_news
    if len(news_list[n]) > 0:
        for j in range(1, int(len(news_list[n])/4+1), 1):
            try:
                print(j)
                get_news(news_list[n][j*4-3], voters.iloc[n, 0], news_list[n][j*4-1], voters.iloc[n, 1])
                gpt_news[n].append(get_completion("gpt-3.5-turbo-16k"))
            except:
                print('retry in 30s')
                get_news(news_list[n][j*4-3], voters.iloc[n, 0], news_list[n][j*4-1], voters.iloc[n, 1])
                gpt_news[n].append(get_completion("gpt-3.5-turbo-16k"))
        print(gpt_news[n])
        print('-------------------------------------------------------')


Stored 'gpt_news' (list)
1
2
['X', 'X']
-------------------------------------------------------
Stored 'gpt_news' (list)
1
2
3
4
["2017.08.21. 충북대는 'K-MOOC'로 개발완료된 '국제개발협력의 이해' 강좌에 대한 최종 결과보고회 및 시연회를 개최한다고 밝혔다. (김상욱 교수는 언급되지 않음)", "2016.08.31. 충북대에서는 김상욱 교수가 '정부 3.0과 대학행정'을 주제로 교직원 대상 교육을 진행했다.", "2021.12.02. 김상욱 교수는 한국정책학회에서 열린 지역화폐 주제 토론회에서 '지역화폐시스템의 구조와 행태'에 관한 발표를 진행하였다.", "2021.12.02. 김상욱 충북대 명예교수는 한국정책학회에서 개최되는 토론회에서 '지역화폐시스템의 구조와 행태'에 대해 발표한다."]
-------------------------------------------------------
Stored 'gpt_news' (list)
Stored 'gpt_news' (list)
1
2
3
4
["2022.06.14. 동아대는 '2022년 SW전문인재양성사업'에 선정되었으며, 김상진 교수가 사업 공동책임자로 참여한다.", "2020.11.24. 동아대 바이오헬스융합연구소에서 '의료/바이오/헬스 분야의 데이터사이언스 기술 적용' 세미나를 개최했다. 김상진 교수는 '유전체 빅데이터의 분석' 주제로 발표했다.", '2023.09.13. X', "2022.06.16. 동아대학교 김상진 교수는 '2022년 SW전문인재양성사업'에서 공동책임을 맡아 부산지역의 디지털금융과 스마트헬스케어 부문 교육과정을 운영한다."]
-------------------------------------------------------
Stored 'gpt_news' (list)
1
2
3
4
['2023.08.31. 경북대 의과대학 김상현 교수는 집먼지진드기에 의해 유발되는 민감

In [27]:
gn_df = pd.DataFrame(gpt_news, columns = ['기사요약_1', '기사요약_2', '기사요약_3', '기사요약_4'])
gn_df

,기사요약_1,기사요약_2,기사요약_3,기사요약_4
0,X,X,None,None
1,2017.08.21. 충북대는 'K-MOOC'로 개발완료된 '국제개발협력의 이해' ...,2016.08.31. 충북대에서는 김상욱 교수가 '정부 3.0과 대학행정'을 주제로...,2021.12.02. 김상욱 교수는 한국정책학회에서 열린 지역화폐 주제 토론회에서 ...,2021.12.02. 김상욱 충북대 명예교수는 한국정책학회에서 개최되는 토론회에서 ...
2,None,None,None,None
3,"2022.06.14. 동아대는 '2022년 SW전문인재양성사업'에 선정되었으며, 김...",2020.11.24. 동아대 바이오헬스융합연구소에서 '의료/바이오/헬스 분야의 데이...,2023.09.13. X,2022.06.16. 동아대학교 김상진 교수는 '2022년 SW전문인재양성사업'에서...
4,2023.08.31. 경북대 의과대학 김상현 교수는 집먼지진드기에 의해 유발되는 민...,2023.02.23. 경북대는 특허청과 한국발명진흥회가 주관하는 '지식재산 전문인력...,2023.09.04. 김상현 교수는 경북대 교수로서 집먼지진드기로 인한 중증 아토피...,2023.08.31. 김상현 교수는 경북대학교 의과대학에서 집먼지진드기에 의해 유발...
...,...,...,...,...
95,None,None,None,None
96,2022.05.10. 김태경 교수는 광운대 경영학과 연구팀과 함께 '머신러닝 기반 ...,X,2022.11.30. 경희대 김태경 교수는 포럼 '원자력과 문학: 문학적 상상과 과...,2022.11.30. 경희대 김태경 교수는 포럼 '원자력과 문학: 문학적 상상과 과...
97,2021.12.30. 충북대 보안컨설팅 융합전공 학생들이 '보건의료 워킹그룹 모의 ...,2020.09.04. 충북대 학생팀인 김태성은 'HIRA 빅데이터 기반 산·학·관 ...,X,2022.09.14. X
98,X,None,None,None


In [28]:
profile_df = profile_df.drop(['이름', '소속', '직위'], axis=1)

In [29]:
test_df = pd.concat([df, gn_df, profile_df], axis = 1, join = 'outer')

In [30]:
test_df = test_df[['이름', '소속', '직위', '핸드폰', '이메일', '세부소속', '위치', '학력', '관심분야', '기타 키워드',
                   '제목_1', '날짜_1', '링크_1', '내용_1', '기사요약_1', '제목_2',
       '날짜_2', '링크_2', '내용_2', '기사요약_2', '제목_3', '날짜_3', '링크_3', '내용_3', '기사요약_3', '제목_4', '날짜_4',
       '링크_4', '내용_4', '기사요약_4']]


In [31]:
test_df = test_df.fillna('X')

In [32]:
test_df.to_excel('크롤링 정보(상위 200).xlsx')

In [33]:
needed = ['소속', '직위', '세부소속', '관심분야', '기타 키워드', '기사요약_1', '기사요약_2', '기사요약_3', '기사요약_4']
y_data = []
for i in range(len(test_df)):
    y_data.append([])

In [35]:
#index에 따라 수정
for i in range(len(test_df)):
    for j in needed:
        y_data[i].append(test_df.loc[i, j])

In [46]:
#요약된 기사들 재요약
#x: 이름, y: 정보(프로필&뉴스기사 요약), z: 학력
def get_newsorg(x, y, z):
    global summarize
    summarize = f"""
        아래의 조건들을 준수하여, 주어진 text로부터 {x}에 대한 정보들을 작성하라.
        1. 주어진 text로부터 {x}의 직함과 업적을 단어의 중복없이 정보의 손실없이, 개조식으로(서술형으로 하지 말고) 요약하라.
        - 직함을 적을 때, '겸임교수', '부교수', '조교수' 등 '교수'의 종류일 경우 구분없이 '교수'로 통일한다
        2. 『』기호 안의 학력이 only '카이스트', 'KAIST', '한국과학기술원', '서울대학교' 4가지 중 하나에 해당할 경우에만, 
            '★' 표시와 함께 그가 해당 학력임을 작성하라.
            The word in『』is his/her academic background. Do not mark '★' if his/her academic background is not '카이스트', 'KAIST',
            '한국과학기술원' or '서울대학교 
        3. {x}가 인공지능 혹은 AI와 관련된 사람일 경우, '☆' 표시와 함께 그가 인공지능 혹은 AI에 관심이 있음을 작성하고,
            가능하다면 그 이유를 작성하라.
            인공지능 혹은 AI와 관련되지 않은 사람일 경우 굳이 문장을 적지 마라. Do not write sentences or '☆' if she/he is not related with AI.
        4. 업적 작성 시, 가능하다면 문장 뒤에 날짜를 기재하라.
        5. 주어진 text만 참고하여 작성하라. 없는 정보를 지어내서는 안 된다. 정보나 업적이 없을 경우 기재하지 않아도 된다.
            [] 기호나 'X'를 써서는 안 된다.
        6. 이름을 적을 필요는 없다.
        7. 아래와 같은 형식에 따라 한국어로 작성하라

        - 
        - 
        - 
        ...

        """
    
    global sum_news
    sum_news = y

    global bg
    bg = z

    global prompt
    prompt = [{"role" : "system",
                "content" : summarize},
                {"role" : "user",
                "content" : f'{sum_news}'},
                {"role" : "user",
                "content" : f'{bg}'}]
    
    return prompt

In [39]:
gpt_final =  []
for i in range(len(test_df)):
    gpt_final.append([])

In [40]:
for n in range(len(test_df)):
    %store gpt_final
    try:
        get_newsorg(voters.iloc[n, 0], y_data[n], f'『{test_df.iloc[n, 7]}』')
        gpt_final[n].append(get_completion("gpt-4"))
        print(gpt_final[n][0])
        print('---------------------------------------------')
    except:
        time.sleep(30)
        get_newsorg(voters.iloc[n, 0], y_data[n], f'『{test_df.iloc[n, 7]}』')
        gpt_final[n].append(get_completion("gpt-4"))
        print(gpt_final[n][0])
        print('---------------------------------------------')

Stored 'gpt_final' (list)
- 교수
- 용인대학교
- 경영학 전공
- 한국인터넷전자상거래학회 추계학술대회 우수논문상
- 한국공정거래조정원 공정거래 자율준수프로그램 등급평가위원
---------------------------------------------
Stored 'gpt_final' (list)
- 교수
- 전자정부
- 전자정부 50년을 빛낸 인물 30선 선정
- 청주지역정보센터(CHAINS) 최우수센터 이끌기
- '정부 3.0과 대학행정' 주제 교직원 대상 교육 진행 (2016.08.31)
- '지역화폐시스템의 구조와 행태' 주제 발표 (2021.12.02)
---------------------------------------------
Stored 'gpt_final' (list)

---------------------------------------------
Stored 'gpt_final' (list)
- 동아대학교 교수
- 빅데이터분석및응용, 의학통계 강의
- 2022년 SW전문인재양성사업 공동책임자, 2022.06.14
- 의료/바이오/헬스 분야의 데이터사이언스 기술 적용 세미나에서 유전체 빅데이터의 분석 주제 발표, 2020.11.24
- 2022년 SW전문인재양성사업에서 부산지역의 디지털금융과 스마트헬스케어 부문 교육과정 운영, 2022.06.16
---------------------------------------------
Stored 'gpt_final' (list)
- 교수
- 약리학
- 집먼지진드기에 의해 유발되는 민감성 중증 만성 아토피피부염의 악성 고리를 담당하는 중요한 인자 발견, 아토피피부염 표적치료제 개발 연구 진행 (2023.08.31)
- '지식재산 전문인력양성 중점대학' 사업단장, 지식재산 전문인력 양성 (2023.02.23)
- 집먼지진드기로 인한 중증 아토피 피부염에 대한 치료법 개발 (2023.09.04)
--------------------------------------------

In [47]:
#index에 따라 수정
adf =pd.DataFrame(gpt_final)
#adf.index = [x for x in range(100, 200 , 1)]

In [48]:
test_df.insert(3, '프로필', adf)

ValueError: cannot insert 프로필, already exists

In [52]:
len(test_df.columns)

31

In [53]:
test_df.to_excel('유권자정보(~200).xlsx')

In [54]:
%store gpt_news
%store gpt_final
%store test_df

Stored 'gpt_news' (list)
Stored 'gpt_final' (list)
Stored 'test_df' (DataFrame)
